In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(rasqualTools))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(parallel))

In [2]:
coln = c('Feature','rsID','Chromosome','position','Ref' ,'Alt' ,'Af','HWEChi_square' ,'IA','Log10_qval','Chi_square',
        'Effect_size','Sequencing_mapping_error_rate','Ref_allele_bias' ,'Overdispersion','SNPid_within_region',
        'No_fSNPs','No_tested_SNPs','No_iterations_for_H0','No_iterations_for_H1',"ties",'Log_likelihood_H0',
        'Convergence_status','r2_fSNPs','r2_rSNP')

In [3]:
rasqual_caQTL = function(x, snp_counts, counts, offsets, covariates,  vcf=vcf_file, outdir, outlogdir,
                         nsamples=10, lead=TRUE, permut=FALSE) 
{
region  = paste0(snp_counts$chromosome_name[x], ":",snp_counts$range_start[x] ,"-",snp_counts$range_end[x])
outfile = paste0(outdir, "/",  snp_counts$gene_id[x])   
outlog    = paste0(outlogdir,"/",  snp_counts$gene_id[x], ".log")
    
system(paste("tabix",  vcf, region, "| $RASQUALDIR/bin/rasqual", 
'-y', counts,
'-k', offsets,
'-x', covariates,
"-n", nsamples,             
'-j', indexes[x] ,
'-l', snp_counts$cis_snp_count[x] ,
'-m', snp_counts$feature_snp_count[x] , 
'-s', snp_counts$exon_starts[x],
'-e', snp_counts$exon_ends[x] , 
'-f', snp_counts$gene_id[x],
 c("","-t" )  [(lead==TRUE) +1]   ,
 c("","-r" )  [(permut==TRUE) +1]   ,
#'--min-coverage-depth 6',
 '--population-only',
'>', outfile, "2>", outlog))
    
}

In [15]:
###### set parmetes here ######
maindir     = '/nfs/lab/projects/pbmc_snATAC/analysis_v2/rasqual_eur/broad/'
celltypes   = list.files(maindir)[!grepl(".txt", list.files(maindir))]
P           =  TRUE ## permutation?  ## not necessary with the new vcf files
folder_name = 'pop1_perm2_lead'
L           =  TRUE ## report only lead?
nsamples    = 10
####################################

setwd(maindir)

In [16]:
celltypes = c('b', 'mono', 'nk', 't')

In [17]:
for (c in celltypes){

cat (c, "\n")
counts      = paste0( c, "/counts.",c, ".bin" )
offsets     = paste0( c, "/size_factors.",c, ".bin" )
#covariates  = paste0( c, "/covariates.",c, ".bin" )
covariates  = paste0( c, "/covariates2.",c, ".bin" )

inp = read.table(paste0( c, "/counts.",c, ".txt" ), row.names=1)
cm  = read.table(paste0( c,"/",c, ".count_matrix" ), header=T, stringsAsFactors = F)

peak_data = cm[,c(1,2,5,3,4)]
peak_data$Strand = as.integer(1)
colnames(peak_data) =  c('gene_id','chr','strand','exon_starts','exon_ends')
peak_data$exon_starts = as.character(peak_data$exon_starts)
peak_data$exon_ends = as.character(peak_data$exon_ends)

vcf_dir = paste0(c, '/vcfs_peaksonly/' )
    
system(paste0("bcftools query -f '%CHROM\\t%POS\\t%ID\\n' ", 
              vcf_dir,  "full.ase.filtered.vcf.gz > ",  vcf_dir, 'full.ase.filtered.snps'))
   
snps2           = read.table(paste0(vcf_dir, "full.ase.filtered.snps"))
colnames(snps2) = c('chr','pos','snp_id')
    
snp_counts2 = countSnpsOverlapingExons(peak_data, snps2, cis_window = 10000)
snp_counts2 = subset(snp_counts2, feature_snp_count>0 |  cis_snp_count> 0)
snp_counts_sub = subset(snp_counts2, gene_id %in% rownames(inp))

indexes =  match ( snp_counts_sub$gene_id,rownames(inp))

cat(sum(rownames(inp[indexes,])!= snp_counts_sub$gene_id), "\n")

sp = split(snp_counts_sub, snp_counts_sub$chromosome_name)

for ( n in 1:22){

chrom  = paste0("chr", n)
outdir = paste( c, folder_name , chrom, sep ="/")
system(paste('mkdir -p', outdir))
outlogdir = paste0(outdir, "/logs")
system(paste("mkdir",outlogdir ))
    
snp_counts_use = sp[[chrom]]
indexes        =  match ( snp_counts_use$gene_id,rownames(inp))

vcf_file = paste0(vcf_dir,  'full.ase.filtered.vcf.gz' )
    

mclapply(1:nrow(snp_counts_use), function(x) rasqual_caQTL(x, snp_counts_use, counts, offsets, covariates, 
                                                           nsamples=nsamples, 
                                                           vcf=vcf_file, outdir=outdir, outlogdir=outlogdir ,
                                                           lead=L, permut=P) , mc.cores = 40 )
         }
    
    }

b 
0 
mono 
0 
nk 
0 
t 
0 


In [18]:
for (c in celltypes){
    logdir = paste0( c, "/",folder_name , "_logs")
    dir.create(logdir)
for ( n in 1:22){
        chrom  = paste0("chr", n)
        outdir = paste( c, folder_name , chrom, "logs",sep ="/")
        files = na.omit(list.files(outdir)[sapply(paste(outdir, list.files(outdir),sep="/"), file.size) >0])
 if(length(files)>0) {      
    error_files= paste(outdir, files, sep="/")
     for (err in error_files){
     system(paste("mv", err, logdir))
    }
     }
    system(paste("rm -r",outdir ))
    
            }
    
    }

In [ ]:
compile_results = function(n,folder_name,outdir ){
  	    chrom  = paste0("chr", n)
        outdir = paste( c, folder_name , chrom, sep ="/")
        system(paste0("cat ", outdir,  "/* > ", c , "/", folder_name,".results_chr", n))
    }


for (c in celltypes){
    cat (c, "\n")
    tabname = paste0(c, "/Results_", folder_name, ".tsv")
    
        
 mclapply (1:22, function(x) compile_results(n=x,folder_name=folder_name,outdir=outdir))
         
        system(paste0("cat ", c , "/", folder_name,".results_chr* > ", tabname))
        system(paste0("rm ", c , "/", folder_name,".results_chr*"))
         results           = read.table(tabname, fill=T)
         colnames(results) = coln
         results           = subset(results,results$rsID !="SKIPPED")
         results$P_VAl     = pchisq(results[,11], 1, lower=F)
         write.table(results, tabname, sep="\t", quote=F)

}


b 
mono 
nk 
t 


In [20]:
c

[1] "t"

In [48]:
# for (c in celltypes){
#     for ( n in 1:22){
#         chrom  = paste0("chr", n)
#         outdir = paste( c, folder_name , chrom, sep ="/")         
#         system(paste0("rm -r ", outdir))
        
#             }
#     system(paste0("rm -r ", c, "/", folder_name))
#  }

### Calculte empirical q-value from the permutations.
see: https://github.com/natsuhiko/rasqual/issues/21

In [21]:
# q1 : real lead Q-value vector for all peaks from RASQUAL
# q0 : permutated Q-value vector
# alpha : FDR threshold
# This function returns the P-value threshold corresponding to FDR=alpha.
getFDR <-
function(q1, q0, alpha=0.1, z=NULL, subset=NULL){
	if(is.null(z)){
		a=0
		for(itr in 1:10){
			a=getFDR(q1,q0,alpha,rev(a+0:100/100^itr),subset)
		}
		a
	}else{
		if(!is.null(subset)){
			q1=q1[subset]
			q0=q0[subset]
		}
		q1=q1[!is.na(q1)]
		q0=q0[!is.na(q0)]
		x=NULL;
		for(i in z){
			x=c(x,sum(q0<i)/length(q0)/(sum(q1<i)/length(q1)))
		};
		max(c(0,z[x<alpha]),na.rm=T)
	}
}


In [22]:
celltypes

[1] "b"    "mono" "nk"   "t"

In [23]:
df = data.frame()
for (c in celltypes) {
results       = read.table(paste0(c, "/Results_pop1_all.tsv"), header=T, stringsAsFactors = F)
random        = read.table(paste0(c, "/Results_pop1_perm1_lead.tsv"), header=T, stringsAsFactors = F)
random2       = read.table(paste0(c, "/Results_pop1_perm2_lead.tsv"), header=T, stringsAsFactors = F)
results       = results[order(results$P_VAl),]
results_lead  = results[!duplicated(results$Feature),]
    
thresh10      = getFDR(10^(results_lead$Log10_qval), 10^(c(random$Log10_qval,random2$Log10_qval )), 0.1) 
thresh05       = getFDR(10^(results_lead$Log10_qval), 10^(c(random$Log10_qval,random2$Log10_qval )), 0.05) 
thresh01       = getFDR(10^(results_lead$Log10_qval), 10^(c(random$Log10_qval,random2$Log10_qval )), 0.01) 

results_lead$flag_fdr10 = 10^(results_lead$Log10_qval) < thresh10
results_lead$flag_fdr05 = 10^(results_lead$Log10_qval) < thresh05
results_lead$flag_fdr01 = 10^(results_lead$Log10_qval) < thresh01
df = rbind(df, c(thresh10, sum(results_lead$flag_fdr10, na.rm=T) , 
                   thresh05, sum(results_lead$flag_fdr05, na.rm=T),
                   thresh01, sum(results_lead$flag_fdr01, na.rm=T)))

write.table(results_lead, paste0(c, "/Results_pop1_lead.tsv"),sep="\t", quote=F, row.names=F)    

}
rownames(df) = celltypes
colnames(df) = c('pval_fdr10%', 'caQTL_fdr10%', 'pval_fdr05%', 'caQTL_fdr05%','pval_fdr01%', 'caQTL_fdr01%')

In [24]:
df

,pval_fdr10%,caQTL_fdr10%,pval_fdr05%,caQTL_fdr05%,pval_fdr01%,caQTL_fdr01%
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
b,0.04530466,110,0.007913403,8,0.007913403,8
mono,0.05061130,404,0.023053811,147,0.006436870,19
nk,0.01363976,17,0.009861591,11,0.005301452,2
t,0.01207348,52,0.008923991,33,0.002886449,5


In [119]:
#write.table(df, "Summary_run3.txt", quote=F)

In [69]:
sum(results_lead$No_fSNPs)

[1] 0